In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 20.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 ht

In [ ]:
# Chunk 2: Importa librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir el path a tu carpeta de trabajo
workdir = '/content/drive/MyDrive/WebScraping Projects/Proyecto de Pricing/Scrapers/epa'

# Cambiar el directorio de trabajo
os.chdir(workdir)
print("Directorio actual:", os.getcwd())

Mounted at /content/drive
Directorio actual: /content/drive/MyDrive/WebScraping Projects/Proyecto de Pricing/Scrapers/epa


In [ ]:
# --------- Chunk 4: Setup driver headless ---------
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()


In [ ]:
# Ruta a la carpeta data donde está el archivo de artículos
data_dir = '/content/drive/MyDrive/WebScraping Projects/Proyecto de Pricing/data'

# Nombre del archivo de entrada
archivo_entrada = 'lista_articulos_2.xlsx'

# Ruta completa al archivo de entrada
ruta_archivo_entrada = os.path.join(data_dir, archivo_entrada)

# Leer archivo de artículos
lista_articulos = pd.read_excel(ruta_archivo_entrada)

In [ ]:
# --------- Chunk: Setup Google Sheets conexión ---------
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Cambia el URL y hoja según tu estructura
sheet_url = 'https://docs.google.com/spreadsheets/d/1Ky1lJhvBP2VntEaLzSTs7KTjMPfxhuB6aFE7KqhYE5w/edit?gid=0'
sh = gc.open_by_url(sheet_url)
worksheet = sh.worksheet("data_extraida")  # <--- aquí el nombre correcto de la hoja

In [ ]:
# --------- Chunk X: Cargar códigos ya existentes para deduplicación ---------
# Lee la columna "codigo" (B) de la hoja y crea un set
codigos_existentes = set(worksheet.col_values(3)[1:])  # [1:] para saltar encabezado

In [ ]:
# --------- Chunk 7: Columnas objetivo y parámetro de testing ---------
columnas = [
    "palabra_key",    # nombre_articulo buscado
    "descripcion",
    "codigo",
    "marca",
    "precio",
    "referencia",
    "barra",
    "url"
]
LIMIT = None  # <-- Cambia para testing, pon None para full luego



In [ ]:
# --------- Chunk 8: Función de extracción detalle ---------
def extraer_detalle_epa(driver, url):
    import time
    from selenium.webdriver.common.by import By

    driver.get(url)
    time.sleep(2)
    # Nombre del producto
    try:
        nombre = driver.find_element(By.CSS_SELECTOR, "h1.page-title span").text.strip()
    except:
        nombre = ""
    # Precio
    try:
        price_elem = driver.find_element(By.CSS_SELECTOR, "span.price")
        price_text = price_elem.text.strip()
        precio = price_text.replace('¢', '').replace(' ', '').replace('.', '')
    except:
        precio = ""
    # Código de producto (nuevo método, rápido y robusto)
    try:
        codigo = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.sku div.value").text.strip()
    except:
        codigo = ""
    # Marca
    marca = "S/M"
    try:
        try:
            tab = driver.find_element(By.ID, "tab-label-additional")
            if "active" not in tab.get_attribute("class"):
                tab.click()
                time.sleep(1)
        except Exception:
            pass
        rows = driver.find_elements(By.CSS_SELECTOR, "#additional table tbody tr")
        for row in rows:
            th = row.find_element(By.CSS_SELECTOR, "th").text.strip().lower()
            td = row.find_element(By.CSS_SELECTOR, "td").text.strip()
            if th == "marca":
                marca = td
    except:
        marca = "S/M"
    referencia = "S/R"
    barra = "S/B"
    return nombre, precio, codigo, marca, referencia, barra


In [ ]:
# --------- Chunk 9: Función para guardar en Google Sheets en tiempo real ---------
def guardar_producto_en_sheets(producto, worksheet):
    fila = [producto.get(col, "") for col in columnas]
    worksheet.append_row(fila, value_input_option="USER_ENTERED")



In [ ]:
# --------- Chunk 10: Loop principal de scraping con guardado en tiempo real ---------
for idx, row in lista_articulos.head(LIMIT).iterrows():
    palabra_key = row['nombre_articulo']
    print(f"Buscando '{palabra_key}' en EPA...")
    # Buscar barra y enviar búsqueda
    EPA_URL = "https://cr.epaenlinea.com/"
    driver.get(EPA_URL)
    time.sleep(2)
    barra = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "aa-Input"))
    )
    barra.clear()
    barra.send_keys(palabra_key)
    barra.send_keys(Keys.ENTER)
    time.sleep(3)
    # Mapear links de resultado
    links = driver.find_elements(By.CSS_SELECTOR, "a.result")
    urls = [link.get_attribute("href") for link in links]
    print(f"  {len(urls)} productos encontrados")
    for url in urls:
        try:
            nombre, precio, codigo, marca, referencia, barra_val = extraer_detalle_epa(driver, url)
            producto = {
                "palabra_key": palabra_key,
                "descripcion": nombre,
                "codigo": codigo,
                "marca": marca,
                "precio": precio,
                "referencia": referencia,
                "barra": barra_val,
                "url": url
            }
            guardar_producto_en_sheets(producto, worksheet)
        except Exception as e:
            print(f"Error extrayendo {url}: {e}")

Buscando 'Aceite para compresor' en EPA...
  1 productos encontrados
Buscando 'Aceite penetrante wd40' en EPA...
  32 productos encontrados
Buscando 'Acido muriatico 1gal' en EPA...
  7 productos encontrados
Buscando 'Acople rápido hembra' en EPA...
  3 productos encontrados
Buscando 'Adaptador redondo plano blanco' en EPA...
  32 productos encontrados
Buscando 'Adaptador universal 2' en EPA...
  2 productos encontrados
Buscando 'Adhesivo en aerosol' en EPA...
  1 productos encontrados
Buscando 'Adhesivo instantáneo 495' en EPA...
  32 productos encontrados
Buscando 'Alambre' en EPA...
  32 productos encontrados
Buscando 'Alero 125 x' en EPA...
  3 productos encontrados
Buscando 'Armario' en EPA...
  32 productos encontrados
Buscando 'Asiento inodoro ' en EPA...
  29 productos encontrados
Buscando 'Aspiradora humedo / seco' en EPA...
  7 productos encontrados
Buscando 'Atornillador' en EPA...
  9 productos encontrados
Buscando 'Balde' en EPA...
  32 productos encontrados
Buscando 'Base